In [1]:
import pickle
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
import numpy as np
import math
import itertools

from test_model import test_model
import parameter_range 
from HELP import *
from datetime import datetime
from pprint import pprint

# Preprocessing

## Biomed

In [2]:
with open("data/biomed/preprocessed_biomed_data.pickle", "rb") as f:
    biomed_x = pickle.load(f)
biomed_x

FileNotFoundError: [Errno 2] No such file or directory: 'data/biomed/preprocessed_biomed_data.pickle'

In [ ]:
with open("data/biomed/y_biomed.pickle", "rb") as f:
    biomed_y = pickle.load(f)
biomed_y

## Fertility


In [ ]:
with open("data/fertility/preprocessed_fertility_data.pickle", "rb") as f:
    fertility_x = pickle.load(f)
fertility_x

In [ ]:
with open("data/fertility/y_fert.pickle", "rb") as f:
    fertility_y = pickle.load(f)
fertility_y

## Wine

https://www.kaggle.com/datasets/shelvigarg/wine-quality-dataset

Abstract: Datasets included, related to red and white Vinho Verde wine samples, from the north of Portugal. The goal is to model wine quality based on physicochemical tests (see [Cortez et al., 2009], [Web Link: http://www3.dsi.uminho.pt/pcortez/wine/]).

Data Set Characteristics: Multivariate

Number of Instances:4898

Number of Attributes: 12

Missing Values: N/A

Source: 
Paulo Cortez, University of Minho, Guimarães, Portugal, http://www3.dsi.uminho.pt/pcortez 
A. Cerdeira, F. Almeida, T. Matos and J. Reis, Viticulture Commission of the Vinho Verde Region(CVRVV), Porto, Portugal


Input variables (based on physicochemical tests):
- fixed acidity
- volatile acidity
- citric acid
- residual sugar
- chlorides
- free sulfur dioxide
- total sulfur dioxide
- density
- pH
- sulfates
- alcohol
Output variable (based on sensory data): 
quality (score between 0 and 10)




winequality.csv

In [3]:
with open("data/wine/winequality.csv", "rb") as f:
    wine = pd.read_csv(f)
wine

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,NaN,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [4]:
# NAs: drop
wine = wine.dropna()
wine.shape

(6463, 13)

In [5]:
# type: 1-hot encoded
wine["white"] = wine.type.apply(lambda x: 1 if x == "white" else 0)
wine["red"] = wine.type.apply(lambda x: 1 if x == "red" else 0)
wine.drop("type", axis=1, inplace=True)
wine

/var/folders/wn/h7d3tkjj12g2j3lhccrbhnlc0000gn/T/ipykernel_77525/1311090816.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine["white"] = wine.type.apply(lambda x: 1 if x == "white" else 0)
/var/folders/wn/h7d3tkjj12g2j3lhccrbhnlc0000gn/T/ipykernel_77525/1311090816.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine["red"] = wine.type.apply(lambda x: 1 if x == "red" else 0)
/var/folders/wn/h7d3tkjj12g2j3lhccrbhnlc0000gn/T/ipykernel_77525/1311090816.py:4: SettingWithCopyWarning: 
A value is trying

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,white,red
0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,1,0
1,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,1,0
2,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,1,0
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,1,0
4,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6491,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,0,1
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,0,1
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,0,1
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,0,1


In [6]:
# separate x and y data
wine_x = wine
wine_y = wine_x.pop("quality")

In [7]:
# scale the data
preprocessor = Pipeline(
   steps=[
       ("scaler", MinMaxScaler())
    ]
)
cols = wine_x.columns
wine_x = pd.DataFrame(preprocessor.fit_transform(wine_x))
wine_x.columns = cols
wine_x


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,white,red
0,0.264463,0.126667,0.216867,0.308282,0.059801,0.152778,0.377880,0.267785,0.217054,0.129213,0.115942,1.0,0.0
1,0.206612,0.146667,0.204819,0.015337,0.066445,0.045139,0.290323,0.132832,0.449612,0.151685,0.217391,1.0,0.0
2,0.355372,0.133333,0.240964,0.096626,0.068106,0.100694,0.209677,0.154039,0.418605,0.123596,0.304348,1.0,0.0
3,0.280992,0.100000,0.192771,0.121166,0.081395,0.159722,0.414747,0.163678,0.364341,0.101124,0.275362,1.0,0.0
4,0.280992,0.100000,0.192771,0.121166,0.081395,0.159722,0.414747,0.163678,0.364341,0.101124,0.275362,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6458,0.247934,0.360000,0.048193,0.019939,0.098007,0.093750,0.073733,0.181222,0.542636,0.337079,0.217391,0.0,1.0
6459,0.198347,0.346667,0.048193,0.021472,0.134551,0.107639,0.087558,0.150183,0.565891,0.202247,0.362319,0.0,1.0
6460,0.206612,0.286667,0.078313,0.026074,0.111296,0.097222,0.078341,0.166377,0.542636,0.297753,0.434783,0.0,1.0
6461,0.173554,0.376667,0.072289,0.021472,0.109635,0.107639,0.087558,0.161172,0.658915,0.275281,0.318841,0.0,1.0


In [8]:
with open("data/wine/wine_x.pickle", "wb") as f:
    pickle.dump(wine_x, f)
with open("data/wine/wine_y.pickle", "wb") as f:
    pickle.dump(wine_y, f)

## Congress

In [18]:

with open("data/congress/congress_x.pickle", "rb") as f:
    congress_x = pickle.load(f)
with open("data/congress/congress_y.pickle", "rb") as f:
    congress_y = pickle.load(f)

congress_x

0      republican
1        democrat
2        democrat
3        democrat
4      republican
          ...    
213      democrat
214      democrat
215    republican
216    republican
217      democrat
Name: class, Length: 218, dtype: object

In [ ]:
congress_y.

# Classifier Prep

In [ ]:
# this is how a testing of a model looks like
input_data = biomed_x
output_data = biomed_y

# get a model
model = MLPClassifier
# prepare the params
params = {
    "hidden_layer_sizes": (4,),
    "activation": "logistic",
    "alpha": ,
    "solver": "adam",
    "learning_rate": "constant",
    "learning_rate_init": 0.05
}
# pass all that and the data to the function test_model
x = test_model(model, params, congress_x, congress_y)
# you get back the means of the results
# you can also specify k as the number of folds
# you can add the argument mean=False to get the k results instead of their means
x

In [ ]:
list(x)

In [ ]:
y = pd.DataFrame(columns = ["a", "b"])
y.loc[len(y.index)] = ['Amy', 93] 

y

# Param Ranges


## Architecture
There are rules of thumb on the architecture of NNs.  
"One hidden layer is sufficient for the large majority of problems." (Taken from : https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw)  
Perhaps we could try adding more for good measure, even though we will probably overfit with our small datasets.  

Here is a set of rules (taken from https://www.linkedin.com/pulse/choosing-number-hidden-layers-neurons-neural-networks-sachdev/)  
- Well if the data is linearly separable then you don't need any hidden layers at all. 
- If data is less complex and is having fewer dimensions or features then neural networks with 1 to 2 hidden layers would work.
- If data is having large dimensions or features then to get an optimum solution, 3 to 5 hidden layers can be used. 



What about layer sizes? Here are some rule suggestions (taken from the same link above):  
- optimal size of the hidden layer is usually between the size of the input and size of the output layers
-  The number of hidden neurons should be between the size of the input layer and the size of the output layer.
-  The number of hidden neurons should be 2/3 the size of the input layer, plus the size of the output layer.
-  The number of hidden neurons should be less than twice the size of the input layer.


Someone on the same forum suggested the following formula for a maximum number of hidden layers:  
$N_h$ = number of hidden layers  
$N_i$ = number of input neurons  
$N_o$ = number of output neurons  
$N_s$ = number of input neurons  
$\alpha$ = number of input neurons  


In [ ]:
input_data = congress_x
output_data = congress_y
# this function gets suggestions for architectures for grid search for example
# n_neurons_steps: Number of steps to take between min and max number of neurons
# even just using 10 steps, the function yields 2350 different architectures, hundreds of which will yield identical results
# but I guess that's the nature of grid searching
parameter_range.get_architecture_ranges(congress_x, congress_y)


# Grid Search

In [ ]:
from gridsearch import grid_search

input_data = congress_x
output_data = congress_y
data_title = "congress"
model = MLPClassifier

grid_search(model=model, 
                input_data=input_data, 
                output_data=output_data, 
                data_title="congress", 
                model_name="MLP",
            skip_til=23000
                )

In [ ]:
for x in range(100):
    print(x, end="\r")